In [1]:
import tensorflow as tf

import os
import sys
import pickle

import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

/home/yolkin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dir_path = os.path.split(os.getcwd())[0]
sys.path.append(dir_path)

In [3]:
embeddings = np.loadtxt(os.path.join(dir_path, 'cache', 'embeddings.vec'))

pickle_in = open(os.path.join(dir_path,'cache', 'database.dict'), 'rb')
db = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open(os.path.join(dir_path,'cache','data2onehot.dict'), 'rb')
dictionary = pickle.load(pickle_in)
pickle_in.close()

In [4]:
# seperate data by length and by each label in one class 'Y_labels'
# and by jump instruction 'Y_if'
#
from JavaClassParser import ByteCode
reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))
X_train=[]
Y_train=[]
X_train_long=[]
Y_train_long=[]
X_labels = []
Y_labels = []

X_if = []
Y_if = []

for dclass in db.values():
    for method in dclass.values():
        instructions = method['x']
        labels = method['y']
        byteIndex = method['index']
        
        
        #
        #seperating by labels

        cur_section = []
        cur_label = labels[0]
        if len(instructions)<100:
            X_train.append(instructions)
            Y_train.append(labels)
        else:
            X_train_long.append(instructions)
            Y_train_long.append(labels)
        for instruction, label in zip(instructions,labels):
            if label != cur_label:
                X_labels.append(cur_section)
                Y_labels.append(cur_label)
                cur_section = []
                cur_label = label
            cur_section.append(dictionary.get(instruction,0))
        
        #print(instructions)
        
        
        #
        #seperating by if jumps
        
        for i,label in zip(range(len(instructions)),labels):
        #index = method[i]
        #instruction = reverse_dictionary[index]
            instruction = instructions[i]
            if 'if' in instruction[:2]:
                #offset = int(reverse_dictionary[method[i+1]])
                offset = int(instructions[i+1])
                j = i
                step = np.sign(offset)
                curr = byteIndex[i]
                end = curr+offset
                section = []
                while(curr!=end):
                    section.append(dictionary.get(instructions[j],0))
                    if instructions[j] in ByteCode.mnemonicMap:
                        #curr += ByteCode.mnemonicMap[instructions[j]].getOpCodeCount() + 1
                        curr = byteIndex[j]
                    j += step
                    if curr>=abs(offset):
                        argCounts = ByteCode.mnemonicMap[instructions[j-step]].argsCount
                        for arg in argCounts:
                            if arg > 0:
                                section.append(dictionary.get(instructions[j],0))
                                j += step
                        break

                if offset<0:
                    section = list(reversed(section))
                    section.append(dictionary.get(instructions[i+1],0))
                
                X_if.append(section)
                Y_if.append(label)
                
        #'''
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_train_long = np.array(X_train_long)
Y_train_long = np.array(Y_train_long)
X_labels = np.array(X_labels)
Y_labels = np.array(Y_labels)
X_if = np.array(X_if)
Y_if = np.array(Y_if)

In [5]:
#output_size = np.max(Y_labels[:,1])+1
output_size = 2
learning_rate = 1e-1
batch_size = 64
hidden_size = 8
'''
num_features depends how do u we want to represent data
we can use w2v embbeding to send dense represetation
#num_features =embeddings.shape[1]
we can use sparse representation which for this example requires over 5000 long one hot vector
we can use sparse representation taking only instruction without arguments lowering one hot to 203 dim

in this simple model dense representation didnt show better result then filtered representation 
with only instructions
'''

#num_features = len(ByteCode.mnemonicMap)
num_features =embeddings.shape[0] + 1
num_epochs = 15

sequence_length = 100

rnn2_graph = tf.Graph()

with rnn2_graph.as_default():
    
    sequence = tf.placeholder(tf.int32,[batch_size, sequence_length])
    labels= tf.placeholder(tf.float64,[batch_size, sequence_length, output_size])
    seq_len = tf.placeholder(tf.int64, [batch_size])
    mask = tf.placeholder(tf.float64, [batch_size, sequence_length, output_size])

    cells_fw = [
        tf.nn.rnn_cell.DropoutWrapper(
        tf.contrib.rnn.BasicLSTMCell(hidden_size,activation=tf.nn.tanh),
        output_keep_prob = 0.8)
        for _ in range(1)]
    
    cells_bw = [
        tf.nn.rnn_cell.DropoutWrapper(
        tf.contrib.rnn.BasicLSTMCell(hidden_size,activation=tf.nn.tanh),
        output_keep_prob = 0.8)
        for _ in range(1)]
    
    W_embed = tf.get_variable(
         'W', shape=(num_features, hidden_size), initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
    
    
    W_1 = tf.Variable(tf.random_normal([1,hidden_size*2, hidden_size*2], dtype=tf.float64))
    b_1 = tf.Variable(tf.random_normal([hidden_size*2],dtype=tf.float64))
    W_2 = tf.Variable(tf.random_normal([1,hidden_size*2, output_size], dtype=tf.float64))
    b_2 = tf.Variable(tf.random_normal([output_size], dtype=tf.float64))

    #initial_state = cell_fw.zero_state(batch_size, dtype=tf.float64)


    #outputs, state = tf.nn.dynamic_rnn(cell, sequence, 
    #        initial_state=initial_state, sequence_length=seq_len)

    embed_input = tf.nn.embedding_lookup(W_embed, sequence, name = "embedded_input")
    
    outputs, states_fw, states_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
        cells_fw = cells_fw,
        cells_bw=cells_bw,
        dtype = tf.float64,
        sequence_length = seq_len,
        inputs = embed_input)
    
    outputs = tf.concat(outputs, 2)
    
    outputs_2 = tf.nn.relu(tf.matmul(outputs, tf.tile(W_1, [tf.shape(outputs)[0],1,1])) + b_1)
    logits  = tf.matmul(outputs_2, tf.tile(W_2, [tf.shape(outputs_2)[0],1,1])) + b_2
    logits = logits*mask
    '''
    for output_batch, label_batch in zip(tf.unstack(outputs, axis=1), tf.unstack(labels, axis=1)):
        for output, label in zip(tf.unstack(output_batch, axis=0), tf.unstack(label_batch, axis=0)):
            output = tf.expand_dims(output, 0)
            label = tf.expand_dims(label, 0)
            logits = tf.matmul(output, W)+ b
            loss += tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=logits)
            correct_predictions += tf.reduce_sum(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1)), tf.float64))
        #incorrect_prediciton += batch_size - tf.reduce_sum(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1)),tf.float64))
    '''
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits = logits)

    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    #accuracy = correct_predictions/tf.reduce_sum(tf.cast(seq_len,tf.float64))
    #acc_val, acc_op = tf.metrics.accuracy(tf.argmax(labels,axis=2), tf.argmax(logits, axis=2))
    diff = batch_size*sequence_length - tf.reduce_sum(tf.cast(tf.equal(tf.argmax(labels, axis =2), tf.argmax(logits, axis=2)), tf.float32))
    nonpadsum = tf.cast(tf.reduce_sum(seq_len),tf.float32)
    accuracy = (nonpadsum- diff)/nonpadsum
    #correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
from Model import data
db = data.Data()
db.loadDataFromJSON()
db.relabelData(overwrite=True)
part_1 = db.getPartition(0.9)
part_2 = db.getPartition(0.1)
dictionary = part_1.getVocabulary()
#X_train, Y_train = data.Data.flattenData(part.X, part.Y, sequence_length)
#X_train, Y_train = part.X, part.Y
#X_train = np.array(X_train)
#Y_train = np.array(Y_train)

In [7]:
"""
N = int(0.9*X_train.shape[0])
print(N)
testN = X_train.shape[0]-N

assert N+testN == X_train.shape[0]
n_steps = N//batch_size

new_Y = []
ln, acsum = 0,0
for y_seq in Y_train:
    y_seq = list(map(lambda x: ((x[0],1) if x[1]==1 else (x[0],0)), y_seq))
    new_Y.append(y_seq)
    acsum += sum(list(map((lambda x:x[1]), y_seq)))
    ln += len(y_seq)
Y_train = new_Y
print('BASELINE:', 1-acsum/ln)
Y_train = np.array(Y_train)

from random import shuffle
indices = np.arange(X_train.shape[0])
shuffle(indices)

X_train, X_test = X_train[indices[:N]], X_train[indices[N:N+testN]]
Y_train, Y_test = Y_train[indices[:N]], Y_train[indices[N:N+testN]]

n_steps_test = int(testN/batch_size)
"""
X_train, Y_train = data.Data.flattenData(part_1.X, part_1.Y, sequence_length)
X_test, Y_test = data.Data.flattenData(part_2.X, part_2.Y, sequence_length)

N=len(X_train)
print(N)
print(part_1.baseline)
print(part_2.baseline)
n_steps = int(len(X_train)/batch_size)
n_steps_test = int(len(X_test)/batch_size)
with tf.Session(graph=rnn2_graph) as session:
    iop = tf.global_variables_initializer()
    loc = tf.local_variables_initializer()
    session.run(iop)
    #session.run(loc)
    for epoch in range(num_epochs):
        acc_sum=0
        batch_index = 0
        train_acc = 0
        div_sum = 0
        for step in range(n_steps):
            cur_batch_size = 0
            batch = []
            batch_Y = []
            sequence_len = []
            
            while(cur_batch_size < batch_size):
                cur_seq = []
                y_cur_seq = []
                for x,y in zip(X_train[batch_index],Y_train[batch_index]):
                    if x in ByteCode.mnemonicMap:
                        cur_seq.append(x)
                        y_cur_seq.append(y)
                    if len(cur_seq) == sequence_length:
                        batch.append(cur_seq)
                        batch_Y.append(y_cur_seq)
                        sequence_len.append(len(y_cur_seq))
                        cur_batch_size += 1
                        cur_seq = []
                        y_cur_seq = []
                if cur_seq:
                    batch.append(cur_seq)
                    batch_Y.append(y_cur_seq)
                    sequence_len.append(len(y_cur_seq))
                    cur_batch_size += 1
                batch_index += 1
            
            data = np.zeros([batch_size, sequence_length])
            filled_labels = np.zeros([batch_size, sequence_length, output_size])
            filled_labels[:,:,0] = 0
            mask_val = np.zeros([batch_size, sequence_length, output_size])
            for i, seq in enumerate(batch_Y):
                for j, label in enumerate(seq):
                    onehot=np.zeros(output_size)
                    onehot[label[1]] = 1
                    filled_labels[i,j,:] = onehot
                    mask_val[i,j,:] = 1
            for i in range(batch_size):
                for j in range(sequence_len[i]):
                    data[i,j] = dictionary.get(batch[i][j],0)+1
            
            feed = {sequence:data, labels:filled_labels, seq_len:sequence_len, mask:mask_val}
            ops = [logits, diff, accuracy, train_step]
            logits_val, diff_val, acc_fixed, _= session.run(ops, feed_dict=feed)
            #acc_fixed = session.run([accuracy], feed_dict={seq_len:sequence_len})
            #session.run(loc)
            train_acc += acc_fixed*np.sum(sequence_len)
            div_sum += np.sum(sequence_len)
            
            #print(acc_fixed)
            #print(diff_val)
            #print(np.sum(filled_labels))
            #print(np.sum(sequence_len))
            #print(np.argmax(filled_labels,axis=2)[0][:sequence_len[0]])
            #print(np.argmax(logits_val,axis=2)[0][:sequence_len[0]])
            '''
            zum = 0
            for i,sl in enumerate(sequence_len):
                zum += np.sum(np.argmax(logits_val[i,sl:,:],axis=1))
            print(zum)
            '''
            print('TRAIN:',train_acc/div_sum)
        

        acc_sum=0
        num=0
        acc_pom=0
        acc_num = 0
            
        batch_index = 0
        for step in range(n_steps_test):
            cur_batch_size = 0
            batch = []
            batch_Y = []
            sequence_len = []
            while(cur_batch_size < batch_size):
                cur_seq = []
                y_cur_seq = []
                for x,y in zip(X_test[batch_index],Y_test[batch_index]):
                    if x in ByteCode.mnemonicMap:
                        cur_seq.append(x)
                        y_cur_seq.append(y)
                    if len(cur_seq) == sequence_length:
                        batch.append(cur_seq)
                        batch_Y.append(y_cur_seq)
                        sequence_len.append(len(y_cur_seq))
                        cur_batch_size += 1
                        cur_seq = []
                        y_cur_seq = []
                if cur_seq:
                    batch.append(cur_seq)
                    batch_Y.append(y_cur_seq)
                    sequence_len.append(len(y_cur_seq))
                    cur_batch_size += 1
                batch_index += 1
                
            data = np.zeros([batch_size, sequence_length])
            filled_labels = np.zeros([batch_size, sequence_length, output_size])
            filled_labels[:,:,0] = 0
            mask_val = np.zeros([batch_size, sequence_length, output_size])
            for i, seq in enumerate(batch_Y):
                for j, label in enumerate(seq):
                    onehot=np.zeros(output_size)
                    onehot[label[1]] = 1
                    filled_labels[i,j,:] = onehot
                    mask_val[i,j,:] = 1
            for i in range(batch_size):
                for j in range(sequence_len[i]):
                    data[i,j] = dictionary.get(batch[i][j],0)+1
            
            feed = {sequence:data, labels:filled_labels, seq_len:sequence_len, mask:mask_val}
            ops = [logits, diff, accuracy]
            logits_val, diff_val, acc_fixed = session.run(ops, feed_dict=feed)
            acc_sum += acc_fixed*np.sum(sequence_len)
            acc_num += np.sum(sequence_len)
            for logit_gold, logit_pred, sl in zip(filled_labels, logits_val, sequence_len):
                acc_pom += np.sum((logit_gold[:sl,1].astype(np.int32)==np.argmax(logit_pred,axis=1)[:sl]))
                num += sl
                #print(logit_gold[:sl,1].astype(np.int32))
                #print(np.argmax(logit_pred,axis=1)[:sl])
                #print(acc_pom/num)
                
            
        print("Epoch %d, step %d ,acc: %f" % (epoch, step, acc_sum/acc_num))
        print(acc_pom/num)
        index = np.argmax(np.sum(np.argmax(filled_labels, axis=2),axis=1))
        print(batch[index])
        print(np.argmax(filled_labels,axis=2)[index][:sequence_len[index]])
        print(np.argmax(logits_val,axis=2)[index][:sequence_len[index]])

13337
0.9253762289643525
0.8913384234211458
TRAIN: 0.8486078977584839
TRAIN: 0.8689414476553291
TRAIN: 0.7169536046220709
TRAIN: 0.7571340513116793
TRAIN: 0.7880443758691338
TRAIN: 0.8071195566557837
TRAIN: 0.8175480761063787
TRAIN: 0.8264226770965902
TRAIN: 0.8336423437972706
TRAIN: 0.8351591821541094
TRAIN: 0.8450936465960196
TRAIN: 0.8474495293290623
TRAIN: 0.8478598118718758
TRAIN: 0.8457758789641446
TRAIN: 0.8479592877716542
TRAIN: 0.8481793803919153
TRAIN: 0.8524568073017036
TRAIN: 0.8520558507979656
TRAIN: 0.8510714444669394
TRAIN: 0.8528617526216702
TRAIN: 0.8522682402724914
TRAIN: 0.8524363254705405
TRAIN: 0.8515791587277389
TRAIN: 0.8529150935381499
TRAIN: 0.8542596557037173
TRAIN: 0.8555005598121532
TRAIN: 0.8571784577852275
TRAIN: 0.8582703570754919
TRAIN: 0.859865740448901
TRAIN: 0.8611519256935658
TRAIN: 0.8620214668329046
TRAIN: 0.8617361571624161
TRAIN: 0.8626780469456924
TRAIN: 0.8634815859806482
TRAIN: 0.8640257774330883
TRAIN: 0.8647426713148645
TRAIN: 0.865575556136

KeyboardInterrupt: 